# Cálculo de Tasa de Defunción seleccionada por zona metropolitana

## **Instrucciones:** 
Cambia el directorio del conjunto de datos y el directorio de trabajo en el primer bloque. Para conocer las opciones de defunciones en lista mexicana debes de revisar elconjunto de defunciones registradas - catalogos - lista_mexicana.csv  
## **Links de descarga:** 
**Estadísticas de Defunciones Registradas:** https://www.inegi.org.mx/programas/edr/#datos_abiertos  
**Crecimiento Demográfico:** https://www.gob.mx/cms/uploads/attachment/file/918028/BD_municipales_portada_regiones_FINAL.pdf  
**Características de Metrópolis:** https://datos.gob.mx/dataset/metropolis_mexico_2020/resource/622497d1-7a92-43e1-8279-e8c286889509


In [ ]:
# ================================================================
# Direcciones generales
# ================================================================

import pandas as pd
from pathlib import Path

# Carpeta donde se encuentran los datos originales de EDR
Dir_Defunciones = Path(
    r"F:\Contenido\Laptop\20_Diciembre2025\ViolenciaFamiliar\Repositorio\Data\conjunto_de_datos_edr2024_csv\conjunto_de_datos"
)
# Nombre del archivo CSV de conjunto de defunciones
ARCHIVO_CONJ_DEF = "conjunto_de_datos_defunciones_registradas24_csv.csv"
LISTA_MEX_OBJETIVO = "49B" 
NOMBRE_INDICADOR = "Accidentes_de_trafico_de_vehiculos_de_motor"

# PARÁMETRO DE ANÁLISIS: "O" para Ocurrencia, "R" para Residencia
CRITERIO_ANALISIS = "O"  # Cambia a "R" según necesites

# carpeta con datos por metrópoli   
Dir_Metropolis = Path(
    r"F:\Contenido\Laptop\20_Diciembre2025\ViolenciaFamiliar\Repositorio\Data\Metropolis"
)

# Nombre de archivos de metrópolis
ARCHIVO_Metropoli_Bridge = "Bridge_ZonasMetro.csv"
ARCHIVO_Carateristicas_Metro = "metropolis_caracteristicas.csv"
ARCHIVO_Proyecciones_Demograficas = "Indicadores_Dem_Mun.csv"

# Carpeta de trabajo del proyecto (por defecto, la carpeta desde donde se ejecuta el script)
DIR_PROYECTO = Path.cwd()

In [3]:

# ================================================================
# Paso 1 - Cálculo de población 2024 por Zona Metropolitana (ZM)
# ================================================================

# -------------------------
# 0) Lectura (solo si aún no existen en memoria)
# -------------------------
# Nota: si ya los leíste en el bloque inicial, puedes comentar esta sección.
df_metro_car = pd.read_csv(Dir_Metropolis / ARCHIVO_Carateristicas_Metro, encoding="utf-8")
df_bridge     = pd.read_csv(Dir_Metropolis / ARCHIVO_Metropoli_Bridge, encoding="utf-8")
df_proy       = pd.read_csv(Dir_Metropolis / ARCHIVO_Proyecciones_Demograficas, encoding="latin1")

# -------------------------
# 1) Estandarización de claves (NO cambiar nombres de columnas)
# -------------------------
# Características de metrópolis
df_metro_car["clave_metropoli"] = df_metro_car["clave_metropoli"].astype(str).str.strip()
df_metro_car["nombre"] = df_metro_car["nombre"].astype(str).str.strip()

# Bridge
df_bridge["Cve_Metro"] = df_bridge["Cve_Metro"].astype(str).str.strip()
df_bridge["Cve_mun"]   = df_bridge["Cve_mun"].astype(str).str.strip().str.zfill(5)

# Proyecciones
df_proy["CLAVE"] = df_proy["CLAVE"].astype(str).str.strip().str.zfill(5)
df_proy["AÑO"]   = pd.to_numeric(df_proy["AÑO"], errors="coerce")
df_proy["POB_MIT_MUN"] = pd.to_numeric(df_proy["POB_MIT_MUN"], errors="coerce")

# -------------------------
# 2) Filtrar municipios/año necesarios (2024) y columnas relevantes
# -------------------------
df_proy_2024 = (
    df_proy.loc[df_proy["AÑO"] == 2024, ["CLAVE", "AÑO", "POB_MIT_MUN"]]
    .dropna(subset=["CLAVE", "POB_MIT_MUN"])
    .copy()
)

# -------------------------
# 3) Join (municipio -> ZM) y agregación por ZM
# -------------------------
# Join bridge (mun->metro) con proyecciones 2024 (mun->pob2024)
df_mun_metro_2024 = df_bridge.merge(
    df_proy_2024,
    left_on="Cve_mun",
    right_on="CLAVE",
    how="left",
    validate="many_to_one"  # cada municipio debe tener máximo un registro 2024
)

# Agregar población por ZM
pob2024_zm = (
    df_mun_metro_2024
    .groupby("Cve_Metro", as_index=False)["POB_MIT_MUN"]
    .sum()
    .rename(columns={"Cve_Metro": "clave_metropoli", "POB_MIT_MUN": "poblacion2024"})
)

# Join final con características de metrópolis (clave_metropoli)
df_zm_pob = df_metro_car.merge(
    pob2024_zm,
    on="clave_metropoli",
    how="left",
    validate="one_to_one"
)

# Normalizar nombres de columnas de salida
df_zm_pob = df_zm_pob.rename(columns={"poblacion": "poblacion2020"})

# Selección final EXACTA de columnas requeridas
df_zm_pob = df_zm_pob[["clave_metropoli", "nombre", "poblacion2020", "poblacion2024"]].copy()

# -------------------------
# 4) Exportar
# -------------------------
salida = DIR_PROYECTO / "Output/ZonaMetro_Pob2024.csv"
df_zm_pob.to_csv(salida, index=False, encoding="utf-8")
salida


WindowsPath('f:/Contenido/Laptop/20_Diciembre2025/ViolenciaFamiliar/Repositorio/Output/ZonaMetro_Pob2024.csv')

In [ ]:
# ================================================================
# Paso 2 - Sumar defunciones y calcular tasa por Zona Metropolitana
# ================================================================

# -------------------------
# 0) Lectura (solo si aún no existen en memoria)
# -------------------------
# Nota: si ya los leíste en el bloque inicial, puedes comentar esta sección.
df_edr = pd.read_csv(
    Dir_Defunciones / ARCHIVO_CONJ_DEF,
    encoding="latin1",
    low_memory=False
)

df_bridge = pd.read_csv(
    Dir_Metropolis / ARCHIVO_Metropoli_Bridge,
    encoding="utf-8"
)

df_pob_metro = pd.read_csv(
    DIR_PROYECTO / "Output/ZonaMetro_Pob2024.csv",
    encoding="utf-8"
)

# -------------------------
# 1) Estandarización de claves 
# -------------------------

# Definición dinámica de columnas según el criterio elegido
if CRITERIO_ANALISIS.upper() == "O":
    col_ent, col_mun = "ent_ocurr", "mun_ocurr"
    print("Analizando por lugar de OCURRENCIA")
else:
    col_ent, col_mun = "ent_resid", "mun_resid"
    print("Analizando por lugar de RESIDENCIA")

# Creación de Cve_mun usando las variables dinámicas
df_edr["ent_std"] = df_edr[col_ent].astype(str).str.zfill(2)
df_edr["mun_std"] = df_edr[col_mun].astype(str).str.zfill(3)
df_edr["Cve_mun"] = df_edr["ent_std"] + df_edr["mun_std"]

# Limpieza de lista mexicana y claves de puente
df_edr["lista_mex"] = df_edr["lista_mex"].astype(str).str.strip().str.upper()
df_bridge["Cve_mun"] = df_bridge["Cve_mun"].astype(str).str.zfill(5)

# -------------------------
# 2) Filtrado flexible según parámetros
# -------------------------

filtro = df_edr["lista_mex"] == str(LISTA_MEX_OBJETIVO).strip().upper()

df_edr_filtrado = df_edr.loc[filtro, ["Cve_mun"]].copy()

# -------------------------
# 3) Agrupar por municipio
# -------------------------
df_def_mun = (
    df_edr_filtrado
    .groupby("Cve_mun", as_index=False)
    .size()
    .rename(columns={"size": "Defunciones"})
)

# -------------------------
# 4) Join municipio → metrópoli
# -------------------------

# Eliminar registros donde el municipio no está especificado o es extranjero
df_def_mun = df_def_mun[~df_def_mun["Cve_mun"].str.endswith("999")]

df_def_metro = df_def_mun.merge(
    df_bridge,
    on="Cve_mun",
    how="left",
    validate="many_to_one"
)

df_def_metro_sum = (
    df_def_metro
    .groupby("Cve_Metro", as_index=False)["Defunciones"]
    .sum()
)

# -------------------------
# 5) Join con población y cálculo de tasa
# -------------------------
df_tasa = df_def_metro_sum.merge(
    df_pob_metro[["clave_metropoli", "poblacion2024"]],
    left_on="Cve_Metro",
    right_on="clave_metropoli",
    how="left",
    validate="one_to_one"
)

df_tasa["Tasa_100k"] = (
    df_tasa["Defunciones"] / df_tasa["poblacion2024"] * 100_000
)

df_tasa = (
    df_tasa
    .rename(columns={"poblacion2024": "Pob2024"})
    [["Cve_Metro", "Pob2024", "Defunciones", "Tasa_100k"]]
)

# -------------------------
# 6) Exportar dinámico
# -------------------------
nombre_salida = f"Tasa_{NOMBRE_INDICADOR}_LM{LISTA_MEX_OBJETIVO}.csv"
ruta_salida = DIR_PROYECTO /"Output"/ nombre_salida

df_tasa.to_csv(ruta_salida, index=False, encoding="utf-8")

ruta_salida


WindowsPath('f:/Contenido/Laptop/20_Diciembre2025/ViolenciaFamiliar/Repositorio/Output/Tasa_Accidentes_de_tráfico_de_vehiculos_de_motor_LM49B.csv')